In [542]:
import os
import io
from gsitk.preprocess.stopwords import StopWordsRemover
# file to store the reviews line by line
# opening and writing files with this syntax to encode with utf-8 and not getting ascii errors
with io.open('imdb_reviews.txt', 'w', encoding='utf8') as f:
    counter = 0
    for review in os.listdir('aclImdb/train/pos'):
        with io.open('aclImdb/train/pos/' + review, 'r', encoding='utf8') as f_review:
        # open the file  with the review
            text = f_review.read()  # file only contain a single line with the review
            text = StopWordsRemover().fit_transform([text.lower()])[0]
            
            f.write(text + '\n')    # write a line in the final file with the review
        #f_review.close()
#         if counter > 4000:
#             break
#         counter += 1

    counter = 0
    for review in os.listdir('aclImdb/train/neg'):  # same for for positive than wth negative
        with io.open('aclImdb/train/neg/' + review, 'r', encoding='utf8') as f_review:
            # open the file  with the review
            text = f_review.read()  # file only contain a single line with the review
            text = StopWordsRemover().fit_transform([text.lower()])[0]
            f.write(text + '\n')  # write a line in the final file with the review
        #f_review.close()
#         if counter > 4000:
#             break
#         counter += 1

KeyboardInterrupt: 

In [ ]:
with io.open('imdb_reviews.txt', 'r', encoding='utf8') as corpus:
    lines = corpus.readlines()
    print(lines[:2])

In [ ]:
 
with io.open('aclImdb/train/pos/' + os.listdir('aclImdb/train/pos')[0], 'r', encoding='utf8') as f_review:
        # open the file  with the review
        text = f_review.read()  # file only contain a single line with the review
        print(text.__class__)
        print(StopWordsRemover().fit_transform([text])[0])

##### Execute 'hatedictionary.py imdb_reviews.txt imdb' before continuing with the next part

# IMDB evaluation

In [ ]:
import json
import pandas as pd
import os
import io
import random
import json
from nltk.corpus import stopwords
import string
import numpy as np
from scipy import stats
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import average_precision_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report
from gsitk.preprocess.stopwords import StopWordsRemover

We are going to create the test dataset. We iterate randdomly through the dataset and take equal number of positive reviews than negative reviews

In [ ]:
n_files = 1000

file_list_pos = random.sample(os.listdir('aclImdb/test/pos'), n_files)  # select x random files
file_list_neg = random.sample(os.listdir('aclImdb/test/neg'), n_files)

Create the groundtruth lists

In [ ]:
list_gt = np.append(np.ones(n_files),np.ones(n_files)*-1)  # ground truth

get the dictionaries

In [ ]:
with open('dictionaries/imdb2_densify.json') as json_file:
    dict_densify = json.load(json_file)
with open('dictionaries/imdb2_socialsent.json') as json_file:
    dict_socialsent = json.load(json_file)

In [ ]:
dict_socialsent

In [ ]:
sorted_keys_socialsent = sorted(dict_socialsent, key=dict_socialsent.get)
sorted_keys_densify= sorted(dict_densify, key=dict_densify.get)


wanted_keys = sorted_keys_socialsent[:3000] + sorted_keys_socialsent[-3000:]
dict_socialsent = dict((k, dict_socialsent[k]) for k in wanted_keys)
wanted_keys_dens = sorted_keys_densify[:3000] + sorted_keys_socialsent[-3000:]
dict_densify = dict((k, dict_densify[k]) for k in wanted_keys_dens)

In [ ]:
wanted_keys = sorted_keys_socialsent[:3000] + sorted_keys_socialsent[-3000:]
dict_socialsent = dict((k, dict_socialsent[k]) for k in wanted_keys)
wanted_keys_dens = sorted_keys_densify[:3000] + sorted_keys_socialsent[-3000:]
dict_densify = dict((k, dict_densify[k]) for k in wanted_keys_dens)

In [ ]:
sorted_keys_socialsent[0:30]

In [ ]:
sorted_keys_socialsent[-30:]

In [ ]:
min(list(dict_densify.values()))

Function to normalize the dictionary between -1 and 1 values

In [ ]:
# def normalize_dict(dictionary):

#     value_list = list(dictionary.values())
#     key_list = list(dictionary.keys())
#     max_list = max(value_list)
#     min_list = min(value_list)
#     a = {key_list[i]: 2 * (value_list[i] - min_list)/(max_list - min_list) -1  for i in range(len(key_list))}
#     return a

In [ ]:
# dict_socialsent = normalize_dict(dict_socialsent)
#dict_densify = normalize_dict(dict_densify)

Read each test positive file and calculate the mean value for both densify and socialsent

list_pos_socialsent = np.zeros(n_files)
list_pos_densify = np.zeros(n_files)
stop_words = set(stopwords.words())   # convert into a set for speed purposes
i = 0
for file in file_list_pos:
    with io.open('aclImdb/test/pos/' + file, 'r', encoding='utf8') as review_file:
        review = review_file.read() # read the file
    words = review.split()
    sum_file_socialsent = 0
    sum_file_densify = 0
    counter_socialsent = 0
    counter_densify = 0
    for word in words:
        if word not in stop_words and word not in string.punctuation:
            if dict_socialsent.get(word) is not None:
                sum_file_socialsent += dict_socialsent.get(word)
                counter_socialsent += 1
            if dict_densify.get(word) is not None:    
                sum_file_densify += dict_densify.get(word)
                counter_densify += 1
    list_pos_socialsent[i] = 1 if sum_file_socialsent / counter_socialsent >= 0.5 else -1
    list_pos_densify[i] = 1 if sum_file_densify / counter_densify > 0 else -1
    i += 1
    
    

same for negative files

list_neg_socialsent = np.zeros(n_files)
list_neg_densify = np.zeros(n_files)
stop_words = set(stopwords.words())   # convert into a set for speed purposes
i = 0
for file in file_list_neg:
    with io.open('aclImdb/test/neg/' + file, 'r', encoding='utf8') as review_file:
        review = review_file.read() # read the file
    words = review.split()
    sum_file_socialsent = 0
    sum_file_densify = 0
    counter_socialsent = 0
    counter_densify = 0
    for word in words:
        if word not in stop_words and word not in string.punctuation:
            if dict_socialsent.get(word) is not None:
                sum_file_socialsent += dict_socialsent.get(word)
                counter_socialsent += 1
            if dict_densify.get(word) is not None:    
                sum_file_densify += dict_densify.get(word)
                counter_densify += 1
    list_neg_socialsent[i] = sum_file_socialsent / counter_socialsent
    list_neg_densify[i] = sum_file_densify / counter_densify
    i += 1
    

Join the positive and negative lists

In [ ]:
list_socialsent = np.append(list_pos_socialsent, list_neg_socialsent)
list_densify = np.append(list_pos_densify, list_neg_densify)

Evaluate with pearson coefficient

In [ ]:
evaluation_socialsent = stats.pearsonr(list_gt, list_socialsent)
evaluation_densify = stats.pearsonr(list_gt, list_densify)

In [ ]:
print('Socialsent: ', evaluation_socialsent)
print('Densify: ', evaluation_densify)

In [ ]:
np.min(list_socialsent)

# this way

##### Logistic regression

In [692]:
files_train_pos = os.listdir('aclImdb/train/pos')
files_train_neg = os.listdir('aclImdb/train/neg')
files_test_pos = os.listdir('aclImdb/test/pos')
files_test_neg = os.listdir('aclImdb/test/neg')

Read every file and save the information in a dataframe

In [693]:
def build_imdb_dataframe(file_list, file_path, label, mode, imdb_list):   # path example: 'aclImdb/train/neg/'
    for file in file_list:  
        with io.open(file_path + file, 'r', encoding='utf8') as file_review:
            review = file_review.read() # read the file
            review = StopWordsRemover().fit_transform([review.lower()])
        dict_aux = {}
        dict_aux['review'] = review[0]
        dict_aux['label'] = label
        dict_aux['set'] = mode    # train or test
        imdb_list.append(dict_aux)
    return imdb_list

The label 1 is a positive review and the label '0' a negative one

In [694]:
imdb_list = []   # store all the information, then it is converted to a dataframe
imdb_list = build_imdb_dataframe(files_train_pos, 'aclImdb/train/pos/', 1, 'train', imdb_list)
imdb_list = build_imdb_dataframe(files_train_neg, 'aclImdb/train/neg/', 0, 'train', imdb_list)
imdb_list = build_imdb_dataframe(files_test_pos, 'aclImdb/test/pos/', 1, 'test', imdb_list)
imdb_list = build_imdb_dataframe(files_test_neg, 'aclImdb/test/neg/', 0, 'test', imdb_list)
imdb_reviews = pd.DataFrame(imdb_list)
imdb_reviews

,review,label,set
0,guy desperate action attempts hit gorgeous gir...,1,train
1,ahista ahista one little small brilliant. star...,1,train
2,"wracked guilt lot things felt apart ledge, ace...",1,train
3,think hollow point funny film good moments nev...,1,train
4,"dana andrews stands ""where sidewalk ends"" 1950...",1,train
...,...,...,...
49995,spoiler pants thinking movie made angry crappe...,0,test
49996,meltdown opens scene scientists preparing cond...,0,test
49997,spoilers<br /><br />this movie action packed; ...,0,test
49998,people say children annoying u think ya little...,0,test


In [960]:
imdb_reviews[imdb_reviews['review'].str.contains("jane fonda")]['review']

321      i'm male, given women's movies, really well do...
790      read comments tomreynolds2004 feel jump here. ...
5541     although like stanley & iris tremendously film...
5912     working-class romantic drama director martin r...
6462     may memorable classic, touching romance import...
6722     'stanley iris' show triumph human spirit. stan...
8363     film special place heart, caught first time, t...
9456     production quality, cast, premise, authentic n...
9586     found so-so romance/drama nice ending generall...
13912    given people involved, hard see movie messed d...
15674    saw capsule comment said "great acting." opini...
17517    watched movie times never really thought funny...
18995    robert deniro plays unbelievably intelligent i...
20477    uncertain make misshapen 2007 dramedy. attempt...
22116    disliked movie numerous reasons. within first ...
23557    georgia rule got one - worst movie ever seen l...
25435    rented "the china syndrome" recently mainly re.

In [961]:
imdb_reviews['review'][321]

"i'm male, given women's movies, really well done special story. personal love jane fonda person one hell fine job, deniro usual superb self. everything well done: acting, directing, visuals, settings, photography, casting. enjoy story real people real love - winner."

Create a text file with a limited number of samples, for train and for test

In [695]:
train_set = imdb_reviews[imdb_reviews['set'] == 'train']

In [696]:
test_set = imdb_reviews[imdb_reviews['set'] == 'test']

In [697]:
imdb_reviews = train_set.append(test_set)

In [698]:
len(train_set)

25000

Save the train to a file

In [700]:
train_set['review'].to_csv('imdb_reviews.txt', sep=' ', index=False, header=False)

###### Classification on unigram

In [707]:
vectorizer = CountVectorizer(min_df=15)
unigram_ocurrence_matrix = vectorizer.fit_transform(imdb_reviews['review'])
#print(vectorizer.get_feature_names())
print(unigram_ocurrence_matrix.toarray().shape)

(50000, 21045)


In [761]:
np.set_printoptions(threshold=1000, edgeitems=10)
print(unigram_ocurrence_matrix.toarray())
print('shape: ', unigram_ocurrence_matrix.toarray().shape)

[[0 0 0 0 0 0 0 0 0 1 ... 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 ... 0 0 0 0 0 0 0 0 0 0]
 ...
 [0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 ... 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0]]
shape:  (50000, 21045)


In [709]:
a = np.sum(unigram_ocurrence_matrix, axis=0)
a.shape

(1, 21045)

In [710]:
np.max(a)
unigram_ocurrence_matrix[imdb_reviews['set'] == 'test']

<25000x21045 sparse matrix of type '<class 'numpy.int64'>'
	with 2450886 stored elements in Compressed Sparse Row format>

In [711]:
from sklearn.linear_model import LogisticRegression
clf_unigram = LogisticRegression(random_state=0, max_iter=8000).fit(unigram_ocurrence_matrix[imdb_reviews['set'] == 'train'], 
                                                                    imdb_reviews[imdb_reviews['set'] == 'train']['label'])

In [712]:
y_score_unigram = clf_unigram.predict(unigram_ocurrence_matrix[imdb_reviews['set'] == 'test'])

In [836]:
precision_unigram = average_precision_score(imdb_reviews[imdb_reviews['set'] == 'test']['label'], y_score_unigram)
print(classification_report(y_score_unigram, imdb_reviews[imdb_reviews['set'] == 'test']['label'] ))
print(stats.pearsonr(y_score_unigram, imdb_reviews[imdb_reviews['set'] == 'test']['label']))

              precision    recall  f1-score   support

           0       0.87      0.86      0.86     12693
           1       0.85      0.87      0.86     12307

    accuracy                           0.86     25000
   macro avg       0.86      0.86      0.86     25000
weighted avg       0.86      0.86      0.86     25000

(0.7220860754797889, 0.0)


###### Logistic regression using the dictionary values

Get the dictionary

In [714]:
# hate
with open('dictionaries/imdb2_densify.json') as json_file:
    dict_densify = json.load(json_file)
with open('dictionaries/imdb2_socialsent.json') as json_file:
    dict_socialsent = json.load(json_file)

In [775]:
# sorted_keys_socialsent = sorted(dict_socialsent, key=dict_socialsent.get)
# sorted_keys_densify= sorted(dict_densify, key=dict_densify.get)


# wanted_keys = sorted_keys_socialsent[:3000] + sorted_keys_socialsent[-3000:]
# dict_socialsent = dict((k, dict_socialsent[k]) for k in wanted_keys)
# wanted_keys_dens = sorted_keys_densify[:3000] + sorted_keys_socialsent[-3000:]
# dict_densify = dict((k, dict_densify[k]) for k in wanted_keys_dens)

In [776]:
print(len(list(dict_socialsent.values())))
print(len(list(dict_densify.values())))

63514
63514


In [717]:
def create_dict_value(data):
    counter = 0

    dict_vector_soc = {}   # socialsent
    dict_vector_dens = {}  # densify
    for word in data['review'].split():
        if dict_socialsent.get(word) is not None:
            dict_vector_soc[word] = dict_socialsent.get(word)
            dict_vector_dens[word] = dict_densify.get(word)
    data['vector_soc'] = dict_vector_soc
    data['vector_dens'] = dict_vector_dens
    return data

In [718]:
imdb_reviews = imdb_reviews.apply(create_dict_value, axis=1)
imdb_reviews

,review,label,set,vector_soc,vector_dens
0,guy desperate action attempts hit gorgeous gir...,1,train,"{'guy': 0.9481098584732381, 'desperate': 0.813...","{'guy': 0.14247038960456848, 'desperate': 0.09..."
1,ahista ahista one little small brilliant. star...,1,train,"{'little': 0.9747448137394689, 'small': 0.9277...","{'little': 0.34360048174858093, 'small': 0.604..."
2,"wracked guilt lot things felt apart ledge, ace...",1,train,"{'wracked': 0.5912880078969615, 'guilt': 0.635...","{'wracked': -0.3791770935058594, 'guilt': 0.01..."
3,think hollow point funny film good moments nev...,1,train,"{'think': 0.9601986716024294, 'hollow': 0.7854...","{'think': 0.16891370713710785, 'hollow': 0.324..."
4,"dana andrews stands ""where sidewalk ends"" 1950...",1,train,"{'dana': 0.5221360341507066, 'andrews': 0.5030...","{'dana': 0.6259295344352722, 'andrews': 0.6521..."
...,...,...,...,...,...
49995,spoiler pants thinking movie made angry crappe...,0,test,"{'spoiler': 0.7246482402453214, 'pants': 0.579...","{'spoiler': -0.10844003409147263, 'pants': -0...."
49996,meltdown opens scene scientists preparing cond...,0,test,"{'meltdown': 0.6610800099231706, 'opens': 0.92...","{'meltdown': -0.5444905161857605, 'opens': 0.4..."
49997,spoilers<br /><br />this movie action packed; ...,0,test,"{'movie': 0.7419759867804996, 'action': 0.9048...","{'movie': 0.3418169915676117, 'action': 0.0604..."
49998,people say children annoying u think ya little...,0,test,"{'people': 0.9262139591498093, 'say': 0.954317...","{'people': 0.12654943764209747, 'say': 0.04381..."


create the vectorizer from the dictionary

In [768]:
v = DictVectorizer(sparse=False)
v_soc = v.fit_transform(imdb_reviews['vector_soc'])
v_dens = v.fit_transform(imdb_reviews['vector_dens'])
print(v_soc.shape)
print(v_dens.shape)

(50000, 58874)
(50000, 58874)


In [814]:
np.set_printoptions(threshold= 10, edgeitems=10)

display(v_soc[1][v_soc[1] > 0])
print('shape: ', v_soc.shape)

array([0.53489828, 0.87679611, 0.75959449, 0.65798654, 0.66686455,
       0.6613475 , 0.95913541, 0.95975107, 0.9264372 , 0.85557129, ...,
       0.95505506, 0.95713479, 0.97358063, 0.78794464, 0.93159836,
       0.94900918, 0.94667836, 0.94449698, 0.84076674, 0.95604063])

shape:  (50000, 58874)


Fit a logistic classifier with training data

In [721]:
clf_soc = LogisticRegression(random_state=0, max_iter = 8000).fit(v_soc[imdb_reviews['set'] == 'train'], imdb_reviews[imdb_reviews['set'] == 'train']['label'])
clf_dens = LogisticRegression(random_state=0, max_iter = 8000).fit(v_dens[imdb_reviews['set'] == 'train'], imdb_reviews[imdb_reviews['set'] == 'train']['label'])

Predict class on test data

In [722]:
y_score_soc = clf_soc.predict(v_soc[imdb_reviews['set'] == 'test'])
y_score_dens = clf_dens.predict(v_dens[imdb_reviews['set'] == 'test'])

In [838]:
report_soc = classification_report(y_score_soc, imdb_reviews[imdb_reviews['set'] == 'test']['label'])
report_dens = classification_report(y_score_dens, imdb_reviews[imdb_reviews['set'] == 'test']['label'])

print('Classification report soc: \n', report_soc)
print(stats.pearsonr(y_score_soc, imdb_reviews[imdb_reviews['set'] == 'test']['label']))
print('Classification report densify: \n',report_dens)
print(stats.pearsonr(y_score_dens, imdb_reviews[imdb_reviews['set'] == 'test']['label']))

Classification report soc: 
               precision    recall  f1-score   support

           0       0.85      0.85      0.85     12565
           1       0.85      0.85      0.85     12435

    accuracy                           0.85     25000
   macro avg       0.85      0.85      0.85     25000
weighted avg       0.85      0.85      0.85     25000

(0.6973694284984095, 0.0)
Classification report densify: 
               precision    recall  f1-score   support

           0       0.85      0.85      0.85     12497
           1       0.85      0.85      0.85     12503

    accuracy                           0.85     25000
   macro avg       0.85      0.85      0.85     25000
weighted avg       0.85      0.85      0.85     25000

(0.7032800202544648, 0.0)


###### Classification on the content of the dictionary

In [962]:
sorted_keys_socialsent = sorted(dict_socialsent, key=dict_socialsent.get)
sorted_keys_densify= sorted(dict_densify, key=dict_densify.get)


wanted_keys = sorted_keys_socialsent[:3000] + sorted_keys_socialsent[-3000:]
dict_socialsent = dict((k, dict_socialsent[k]) for k in wanted_keys)
wanted_keys_dens = sorted_keys_densify[:3000] + sorted_keys_densify[-3000:]
dict_densify = dict((k, dict_densify[k]) for k in wanted_keys_dens)

In [924]:
print('Negative values: \n', wanted_keys[:50])
print('Positive values: \n', wanted_keys[-50:])

Negative values: 
 ['disgusting', 'horrible', 'unpleasant', 'horrendous', 'senseless', 'appalling', 'shameful', 'vile', 'disgraceful', 'hideous', 'horrific', 'tragic', 'ghastly', 'unspeakable', 'unfortunate', 'terrible', 'atrocious', 'horrors', 'horrifying', 'sickening', 'dreadful', 'revolting', 'deplorable', 'abominable', 'cowardly', 'terrifying', 'barbaric', 'despicable', 'frightening', 'scandalous', 'barbarous', 'unforgivable', 'odious', 'heinous', 'abhorrent', 'nightmare', 'dastardly', 'reprehensible', 'inexcusable', 'unimaginable', 'shocking', 'horrid', 'grisly', 'repugnant', 'inhuman', 'contemptible', 'grotesque', 'nauseating', 'tragedy', 'gruesome']
Positive values: 
 ['blind', 'remembered', 'likes', 'dreams', 'friends', 'beautiful', 'dad', 'best', 'fine', 'choice', 'wise', 'beloved', 'grateful', 'reminds', 'remembers', 'loves', 'life', 'job', 'lot', 'great', "'m", 'way', 'easy', 'course', 'plenty', 'better', 'makes', 'simple', 'big', 'always', 'remember', 'little', 'touch', 'li

In [963]:
print('Negative values: \n', wanted_keys_dens[:50])
print('Positive values: \n', wanted_keys_dens[-50:])

Negative values: 
 ['lynching', 'usury', 'anti-black', 'beetleborgs', 'anti-catholic', 'ooops', 'discriminates', 'drug-related', 'ouch', 'qld', 'loek', 'violators', 'lynchings', 'puking', 'perpetuated', 'anti-serb', 'vd', 'jayhawkers', 'scrooged', 'wildside', 'thuggery', '1981-82', 'sweatshops', 'pshaw', 'linette', 'dept', 'stereotyping', 'excesses', 'trivialising', 'miscegenation', 'noooo', 'littering', 'ewww', 'sedition', 'rampant', 'afoul', 'sheb', 'profiteering', 'mccarthyism', 'commercialism', 'gouging', 'editorializing', 'blasphemy', 'deplorably', 'overreacting', '______', 'curbed', 'hooliganism', 'feder', 'mongering']
Positive values: 
 ['summit', 'accompany', 'tour', 'beautiful', 'elegant', 'alongside', 'deliver', 'gives', 'airy', 'presentation', 'discussions', 'inspiration', 'host', 'quest', 'splendid', 'wonderful', 'chosen', 'pairing', 'arrangements', 'lovely', 'arrival', 'stages', 'distant', 'offering', 'arrive', 'travels', 'partners', 'arrives', 'visit', 'exploring', 'invit

In [906]:
def evaluate(data):
    evaluation_soc = 0  # socialsent
    evaluation_dens = 0   # densify
    counter_soc = 0
    counter_dens = 0
    for word in data['review'].split():
        if dict_socialsent.get(word) is not None:
            evaluation_soc += dict_socialsent.get(word)
            counter_soc += 1
        if dict_densify.get(word) is not None:    
            evaluation_dens += dict_densify.get(word)
            counter_dens += 1
    if counter_soc == 0:
        counter_soc = 1   # avoid division by 0
        counter_dens = 1
    data['evaluation_soc_all'] = evaluation_soc/counter_soc
    data['evaluation_dens_all'] = evaluation_dens/counter_dens
    data['evaluation_soc'] = 1 if evaluation_soc/counter_soc >= 0.82 else 0
    data['evaluation_dens'] = 1 if evaluation_dens/counter_dens > 0.25 else 0
    return data    

In [907]:
imdb_reviews_evaluated = imdb_reviews[imdb_reviews['set'] == 'test'].apply(evaluate, axis=1)
imdb_reviews_evaluated

,review,label,set,vector_soc,vector_dens,evaluation_soc_all,evaluation_dens_all,evaluation_soc,evaluation_dens
25000,previously seen short vhs tape feature summer ...,1,test,"{'previously': 0.9273066290104699, 'seen': 0.9...","{'previously': 0.2852485775947571, 'seen': 0.2...",0.924398,0.367772,1,1
25001,one successful shows television history back. ...,1,test,"{'successful': 0.9423904039562421, 'shows': 0....","{'successful': 0.6003767848014832, 'shows': 0....",0.926901,0.343104,1,1
25002,"remember film,it first film watched cinema pic...",1,test,"{'remember': 0.9745676232808507, 'first': 0.90...","{'remember': 0.13504120707511902, 'first': 0.5...",0.921300,0.266118,1,1
25003,"salinger's ""a perfect day bananafish"" ends sui...",1,test,"{'perfect': 0.9878818466695282, 'day': 0.90316...","{'perfect': 0.6446346044540405, 'day': 0.39365...",0.909428,0.310720,1,1
25004,"first saw movie video store and, bam margera f...",1,test,"{'first': 0.9055403602457166, 'saw': 0.9339838...","{'first': 0.5304829478263855, 'saw': 0.1342318...",0.896786,0.226796,1,0
...,...,...,...,...,...,...,...,...,...
49995,spoiler pants thinking movie made angry crappe...,0,test,"{'spoiler': 0.7246482402453214, 'pants': 0.579...","{'spoiler': -0.10844003409147263, 'pants': -0....",0.896641,0.280523,1,1
49996,meltdown opens scene scientists preparing cond...,0,test,"{'meltdown': 0.6610800099231706, 'opens': 0.92...","{'meltdown': -0.5444905161857605, 'opens': 0.4...",0.897780,0.283988,1,1
49997,spoilers<br /><br />this movie action packed; ...,0,test,"{'movie': 0.7419759867804996, 'action': 0.9048...","{'movie': 0.3418169915676117, 'action': 0.0604...",0.869731,0.275286,1,1
49998,people say children annoying u think ya little...,0,test,"{'people': 0.9262139591498093, 'say': 0.954317...","{'people': 0.12654943764209747, 'say': 0.04381...",0.834347,0.248250,1,0


In [908]:
imdb_reviews_evaluated['evaluation_dens'].describe()

count    25000.00000
mean         0.87548
std          0.33018
min          0.00000
25%          1.00000
50%          1.00000
75%          1.00000
max          1.00000
Name: evaluation_dens, dtype: float64

get statts

In [909]:
eval_socialsent = stats.pearsonr(imdb_reviews_evaluated['evaluation_soc'], imdb_reviews_evaluated['label'])
eval_densify = stats.pearsonr(imdb_reviews_evaluated['evaluation_dens'], imdb_reviews_evaluated['label'])
print('Pearson socialsent: ', eval_socialsent)
print('Pearson densify: ', eval_densify)

report_soc = classification_report(imdb_reviews_evaluated['evaluation_soc'], imdb_reviews_evaluated['label'])
report_dens = classification_report(imdb_reviews_evaluated['evaluation_dens'], imdb_reviews_evaluated['label'])

print('Classification report soc: \n', report_soc)
print('Classification report densify: \n',report_dens)

Pearson socialsent:  (0.12891610246013305, 4.164848938816207e-93)
Pearson densify:  (0.19420072327819932, 5.4140821374103835e-211)
Classification report soc: 
               precision    recall  f1-score   support

           0       0.07      0.81      0.13      1062
           1       0.98      0.51      0.67     23938

    accuracy                           0.53     25000
   macro avg       0.53      0.66      0.40     25000
weighted avg       0.94      0.53      0.65     25000

Classification report densify: 
               precision    recall  f1-score   support

           0       0.19      0.76      0.30      3113
           1       0.94      0.54      0.68     21887

    accuracy                           0.56     25000
   macro avg       0.56      0.65      0.49     25000
weighted avg       0.85      0.56      0.64     25000



In [910]:
print(imdb_reviews_evaluated[24990:24991][['review', 'vector_soc']]['review'][49990])
print(imdb_reviews_evaluated[24990:24991][['review', 'vector_soc']]['vector_soc'][49990])
print(imdb_reviews_evaluated[24990:24991][['review', 'vector_dens']]['vector_dens'][49990])

sometimes intentionally campy, times unintentionally silly (like opening scene, woman "informed" raped family slaughtered, sake exposition), film ultimately neither funny enough competent enough (as straightforward adventure story) really enjoyable. i'll leave decide may highlight, low point probably fight silly metallic dragon. brigitte nielsen good choice sonja, fresh face firm, slightly muscular body (and think dark-red hair suits perfectly), schwarzenegger gives one colorless performances calidor. (*1/2)
{'sometimes': 0.9501110285917846, 'intentionally': 0.6920996939485773, 'times': 0.9258533604134269, 'unintentionally': 0.5723260764729114, 'silly': 0.2725723304363115, 'opening': 0.905864061607587, 'woman': 0.8645266773118258, 'raped': 0.6073616131473395, 'family': 0.9007768166762729, 'sake': 0.90133546709098, 'film': 0.7076635137747269, 'ultimately': 0.9508800936015241, 'neither': 0.9558237542679288, 'funny': 0.48189733365390325, 'enough': 0.9594047226864929, 'competent': 0.702543

In [911]:
imdb_reviews_evaluated[imdb_reviews_evaluated['evaluation_soc'] == 0]

,review,label,set,vector_soc,vector_dens,evaluation_soc_all,evaluation_dens_all,evaluation_soc,evaluation_dens
25179,mighty like moose one many short films directo...,1,test,"{'mighty': 0.7312970324769965, 'like': 0.96620...","{'mighty': 0.13502411544322968, 'like': 0.3380...",0.807178,0.336687,0,1
25220,three outstanding bbc television series' chris...,1,test,"{'three': 0.6058560404992323, 'outstanding': 0...","{'three': 0.44054365158081055, 'outstanding': ...",0.809603,0.367903,0,1
25230,amazing movie clever using actors sets. also s...,1,test,"{'amazing': 0.7880371402648724, 'movie': 0.741...","{'amazing': 0.509014368057251, 'movie': 0.3418...",0.791260,0.296607,0,1
25320,"sell dead (2009) **1/2 dominic monaghan, larry...",1,test,"{'sell': 0.817990609241611, 'dead': 0.87188900...","{'sell': 0.3423904776573181, 'dead': 0.2134975...",0.747936,0.211122,0,0
25353,hunt justice setup slobadon milosevic trial ha...,1,test,"{'hunt': 0.6210317587919527, 'justice': 0.8084...","{'hunt': 0.36053332686424255, 'justice': -0.14...",0.814706,0.279075,0,1
...,...,...,...,...,...,...,...,...,...
49900,oh. good. grief.<br /><br />i saw movie title ...,0,test,"{'saw': 0.9339838965670254, 'movie': 0.7419759...","{'saw': 0.13423189520835876, 'movie': 0.341816...",0.797076,0.309812,0,1
49937,"los angeles, alcoholic lazy hank chinaski (mat...",0,test,"{'alcoholic': 0.6243188209094036, 'lazy': 0.49...","{'alcoholic': -0.32522428035736084, 'lazy': -0...",0.813735,0.351785,0,1
49944,"script, story, mess!",0,test,{},{},0.000000,0.000000,0,0
49945,"terrible movie, waste money it. even watch fre...",0,test,"{'terrible': 0.07256369814975122, 'waste': 0.6...","{'terrible': -0.37290361523628235, 'waste': -0...",0.713271,0.216330,0,0


In [945]:
#['love', 'loved', 'loves', 'awesome', 'nice', 'amazing', 'best', 'fantastic', 'correct', 'happy']
#>>> neg_seeds
#['hate', 'hated', 'hates', 'terrible', 'nasty', 'awful', 'worst', 'horrible', 'wrong', 'sad']
dict_socialsent.get('sad')

0.182567049375833

In [919]:
np.mean(imdb_reviews_evaluated[imdb_reviews_evaluated['label'] == 0]['evaluation_soc_all'])

0.30070081916310654

In [914]:
np.min(imdb_reviews_evaluated[imdb_reviews_evaluated['label'] == 0]['evaluation_dens_all'])

-0.07578752934932709

In [915]:
dict_socialsent.get(sorted_keys_socialsent[2999])

0.4907470460663988

In [916]:
sorted_keys_socialsent[3000]

'sampling'

6000